Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
from nltk.stem import WordNetLemmatizer

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [53]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [7]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [54]:
tweets = train_df['text'].tolist()

**Analizador de sentimentos pré-treinado do NLTK (VADER)**

In [9]:
sia = SentimentIntensityAnalyzer()

In [10]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(train_df[train_df["target"] == 1]["text"].values[1])

{'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.34}

In [11]:
sia.polarity_scores(train_df[train_df["target"] == 0]["text"].values[1])

{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

In [55]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

In [56]:
sentimento = []
correct = 0
num = 0

In [57]:
for tweet in tweets:
    sentimento.append(is_positive(tweet))
    if is_positive(tweet) == True and train_df['target'][num] == 1:
        correct+=1
    elif is_positive(tweet) == False and train_df['target'][num] == 0:
        correct+=1
    else:
        pass
    num +=1

In [58]:
print(correct*100 / len(tweets))

45.96085642979115


In [63]:
import operator

In [65]:
train_df['sentimento'] = list(map(operator.not_, sentimento))

In [68]:
train_df_sentimento = train_df

In [69]:
train_df_sentimento.head()

,id,keyword,location,text,target,sentimento
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,False
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,True
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,True
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,True
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,True


In [20]:
len(train_df)

3263

In [72]:
filtro = train_df['keyword'].notnull()

In [73]:
filtro

0       False
1       False
2       False
3       False
4       False
        ...  
7608    False
7609    False
7610    False
7611    False
7612    False
Name: keyword, Length: 7613, dtype: bool

In [74]:
train_df_pal=train_df[filtro]

In [75]:
train_df_pal

,id,keyword,location,text,target,sentimento
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,True
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,True
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,True
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,True
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,True
...,...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,True
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,False
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,True
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,False


In [37]:
keyword_unique = train_df_pal['keyword'].unique()

In [39]:
keyword_unique

array(['ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'derailed

In [77]:
train_df_pal['keyword'] = [x.replace('%20', ' ') for x in train_df_pal['keyword']]

['ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'ablaze',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'accident',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',
 'aftershock',

In [49]:
keywords = [x.replace('%20', ' ') for x in keyword_unique]

In [76]:
keyword_sample = keywords[0]
keyword_sample

'ablaze'

In [78]:
sample_df = train_df_pal[train_df_pal.keyword == keyword_sample]
sample_df

,id,keyword,location,text,target,sentimento
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,True
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,True
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,True
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,True
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,True
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0,True
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1,True
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,1,True
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0,False
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0,True


# Lista de tarefas: 
- Limpeza de dados: limpeza de caracteres do Keyword e text
- Estudar 

**Personalizando a análise de sentimento do NLTK**

In [65]:
unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])

def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

#Fazer uma lista das palavras positivas e negativas!!

In [ ]:
positive = []
negative = []

In [ ]:
for t in tweets:
    pal =[]
    pal = word_tokenize(t)
    

In [64]:
def extract_features(text):
    features = dict()
    wordcount = 0
    compound_scores = list()
    positive_scores = list()
    for sentence in tweets:
        for word in nltk.word_tokenize(sentence):
            if word.lower() in top_100_positive:
                wordcount += 1
        compound_scores.append(sia.polarity_scores(sentence)["compound"])
        positive_scores.append(sia.polarity_scores(sentence)["pos"])

In [ ]:
for sentence in tweets:
    for word in nltk.word_tokenize(sentence):
        if word.lower() in top_100_positive:
            wordcount += 1
    compound_scores.append(sia.polarity_scores(sentence)["compound"])
    positive_scores.append(sia.polarity_scores(sentence)["pos"])